In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

In [2]:
list = [['A','B','C','col2,col3','X,Y'],
       ['P','Q','R','col1','Z'],
       ['I','J','K','col1,col3','S,T']]
df=spark.createDataFrame(list,['col1','col2','col3','col4','col5'])
df.show()

+----+----+----+---------+----+
col1|col2|col3| col4|col5|
+----+----+----+---------+----+
 A| B| C|col2,col3| X,Y|
 P| Q| R| col1| Z|
 I| J| K|col1,col3| S,T|
+----+----+----+---------+----+

In [3]:
df=df.withColumn("col6", F.array(F.split(("col4"),','),F.split(("col5"),','))).drop("col4","col5")


In [4]:
from pyspark.sql.functions import udf


@udf("map<string,string>")
def as_dict(x):
    return dict(zip(*x)) if x else None

df.withColumn("col6", F.array(F.split(("col4"),','),F.split(("col5"),','))).drop("col4","col5")\
.withColumn("col6", as_dict("col6")).select("col1","col2","col3",F.col("col6.col1").alias("col1_"),F.col("col6.col2").alias("col2_"),F.col("col6.col3").alias("col3_"))\
.withColumn("col1", F.when(F.col("col1_").isNotNull(), F.col("col1_")).otherwise(F.col("col1")))\
.withColumn("col2", F.when(F.col("col2_").isNotNull(),F.col("col2_")).otherwise(F.col("col2")))\
.withColumn("col3",F.when(F.col("col3_").isNotNull(),F.col("col3_")).otherwise(F.col("col3")))\
.drop("col1_","col2_","col3_")\
.show()

+----+----+----+
col1|col2|col3|
+----+----+----+
 A| X| Y|
 Z| Q| R|
 S| J| T|
+----+----+----+

In [5]:
df.withColumn("col6", F.map_from_arrays(F.split("col4",','),F.split("col5",','))).drop("col4","col5")\
.select("col1","col2","col3",F.col("col6.col1").alias("col1_"),F.col("col6.col2").alias("col2_"),F.col("col6.col3").alias("col3_"))\
.withColumn("col1", F.when(F.col("col1_").isNotNull(), F.col("col1_")).otherwise(F.col("col1")))\
.withColumn("col2", F.when(F.col("col2_").isNotNull(),F.col("col2_")).otherwise(F.col("col2")))\
.withColumn("col3",F.when(F.col("col3_").isNotNull(),F.col("col3_")).otherwise(F.col("col3")))\
.drop("col1_","col2_","col3_")\
.show()


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o2813.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve '`col4`' given input columns: [col1, col2, col3, col6];;
'Project [col1#445, col2#446, col3#447, map_from_arrays(split('col4, ,), split('col5, ,)) AS col6#590]
+- Project [col1#445, col2#446, col3#447, col6#497]
 +- Project [col1#445, col2#446, col3#447, col4#448, col5#449, array(split(col4#448, ,), split(col5#449, ,)) AS col6#497]
 +- LogicalRDD [col1#445, col2#446, col3#447, col4#448, col5#449], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:120)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPla

In [7]:

df.withColumn("col6", F.map_from_arrays(F.split("col4",','),F.split("col5",','))).drop("col4","col5")\
.select("col1","col2","col3",F.col("col6.col1").alias("col1_"),F.col("col6.col2").alias("col2_"),F.col("col6.col3").alias("col3_"))\
.withColumn("col1", F.when(F.col("col1_").isNotNull(), F.col("col1_")).otherwise(F.col("col1")))\
.withColumn("col2", F.when(F.col("col2_").isNotNull(),F.col("col2_")).otherwise(F.col("col2")))\
.withColumn("col3",F.when(F.col("col3_").isNotNull(),F.col("col3_")).otherwise(F.col("col3")))\
.drop("col1_","col2_","col3_")\
.show()



+----+----+----+
col1|col2|col3|
+----+----+----+
 A| X| Y|
 Z| Q| R|
 S| J| T|
+----+----+----+

In [8]:
%scala
val df = Seq(("A","B","C","col2,col3","X,Y"),
       ("P","Q","R","col1","Z"),
       ("I","J","K","col1,col3","S,T")).toDF("col1","col2","col3","col4","col5")
df.show()


import org.apache.spark.sql.functions.{col, map_from_arrays, split, when}

df.withColumn("col6", map_from_arrays(split($"col4",","),split($"col5",","))).drop("col4","col5")
.select($"col1",$"col2",$"col3",col("col6.col1").alias("col1_"),col("col6.col2").alias("col2_"),col("col6.col3").alias("col3_"))
.withColumn("col1", when(col("col1_").isNotNull, col("col1_")).otherwise($"col1"))
.withColumn("col2", when(col("col2_").isNotNull,col("col2_")).otherwise($"col2"))
.withColumn("col3",when(col("col3_").isNotNull,col("col3_")).otherwise($"col3"))
.drop("col1_","col2_","col3_")
.show()

+----+----+----+---------+----+
col1|col2|col3| col4|col5|
+----+----+----+---------+----+
 A| B| C|col2,col3| X,Y|
 P| Q| R| col1| Z|
 I| J| K|col1,col3| S,T|
+----+----+----+---------+----+

+----+----+----+
col1|col2|col3|
+----+----+----+
 A| X| Y|
 Z| Q| R|
 S| J| T|
+----+----+----+

df: org.apache.spark.sql.DataFrame = [col1: string, col2: string ... 3 more fields]
import org.apache.spark.sql.functions.{col, map_from_arrays, split, when}

In [9]:
list=  [['A','X','Y'],         
        ['Z','Q','R'],       
        ['S','J','T']]

df=spark.createDataFrame(list,['col1','col2','col3'])
df.show()

+----+----+----+
col1|col2|col3|
+----+----+----+
 A| X| Y|
 Z| Q| R|
 S| J| T|
+----+----+----+

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

In [11]:
list=[[1,'r1',{"application":{"component":[{"name": "tool", "version": "1.0"}, {"name": "app", "version": "1.0"}]}}],
      [1,'r2',{"application":{"component":[{"name": "tool", "version": "2.0"}, {"name": "app", "version": "2.0"}]}}],
      [2,'r3',{"application":{"component":[{"name": "tool", "version": "3.0"}, {"name": "app", "version": "3.0"}]}}]]
df=spark.createDataFrame(list,['accountId','resourceId','items'])
df.show(truncate=False)

In [12]:
list=[[1,'r1',{"application":{"component":[{"name": "tool", "version": "1.0"}, {"name": "app", "version": "1.0"}]}}],
      [1,'r2',{"application":{"component":[{"name": "tool", "version": "2.0"}, {"name": "app", "version": "2.0"}]}}],
      [2,'r3',{"application":{"component":[{"name": "tool", "version": "3.0"}, {"name": "app", "version": "3.0"}]}}]]
df=spark.createDataFrame(list,['accountId','resourceId','items'])
df.show(truncate=False)

+---------+----------+-----------------------------------------------------------------------------------------------+
accountId|resourceId|items |
+---------+----------+-----------------------------------------------------------------------------------------------+
1 |r1 |[application -> [component -> [[name -> tool, version -> 1.0], [name -> app, version -> 1.0]]]]|
1 |r2 |[application -> [component -> [[name -> tool, version -> 2.0], [name -> app, version -> 2.0]]]]|
2 |r3 |[application -> [component -> [[name -> tool, version -> 3.0], [name -> app, version -> 3.0]]]]|
+---------+----------+-----------------------------------------------------------------------------------------------+

In [13]:
df.printSchema()

root
-- accountId: long (nullable = true)
-- resourceId: string (nullable = true)
-- items: map (nullable = true)
 |-- key: string
 |-- value: map (valueContainsNull = true)
 | |-- key: string
 | |-- value: array (valueContainsNull = true)
 | | |-- element: map (containsNull = true)
 | | | |-- key: string
 | | | |-- value: string (valueContainsNull = true)

In [14]:
display(df)

accountId,resourceId,items
1,r1,"Map(application -> Map(component -> List(Map(name -> tool, version -> 1.0), Map(name -> app, version -> 1.0))))"
1,r2,"Map(application -> Map(component -> List(Map(name -> tool, version -> 2.0), Map(name -> app, version -> 2.0))))"
2,r3,"Map(application -> Map(component -> List(Map(name -> tool, version -> 3.0), Map(name -> app, version -> 3.0))))"


In [15]:
from pyspark.sql import functions as F
df.select("accountId","resourceId", F.explode(F.col("items.application"))).withColumn("value", F.explode("value")).select("accountId","resourceId","value.name","value.version").show()

+---------+----------+----+-------+
accountId|resourceId|name|version|
+---------+----------+----+-------+
 1| r1|tool| 1.0|
 1| r1| app| 1.0|
 1| r2|tool| 2.0|
 1| r2| app| 2.0|
 2| r3|tool| 3.0|
 2| r3| app| 3.0|
+---------+----------+----+-------+

In [16]:
df.withColumn("items", F.explode(F.col("items.application.component"))).select("accountId","resourceId","items.name","items.version").show()

+---------+----------+----+-------+
accountId|resourceId|name|version|
+---------+----------+----+-------+
 1| r1|tool| 1.0|
 1| r1| app| 1.0|
 1| r2|tool| 2.0|
 1| r2| app| 2.0|
 2| r3|tool| 3.0|
 2| r3| app| 3.0|
+---------+----------+----+-------+

In [17]:
df.select("items.*")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o3642.select.
: org.apache.spark.sql.AnalysisException: Can only star expand struct data types. Attribute: `ArrayBuffer(items)`;
	at org.apache.spark.sql.catalyst.analysis.UnresolvedStar.expand(unresolved.scala:321)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$buildExpandedProjectList$1.apply(Analyzer.scala:1045)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$buildExpandedProjectList$1.apply(Analyzer.scala:1043)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.flatMap(TraversableLike.scala:241)
	at scala.collection.AbstractTraversable.flatMap(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$buildExpandedProjectList(Analyzer.scala:1043)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9.applyOrElse(Analyzer.scala:953)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9.applyOrElse(Analyzer.scala:948)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:89)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.apply(Analyzer.scala:948)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.apply(Analyzer.scala:775)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apache.s

In [18]:
root
 |-- accountId: 
 |-- resourceId: 
 |-- PeriodId: 
 |-- items: 
 |    |-- application: 
 |    |    |-- component: array

In [19]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

list=[[1,None,None],
       [2,'2',None],
        [3,None,None],
        [5, None, None],
          [4,'four', 'IV']]

df=spark.createDataFrame(list,['VendorID','col2','col3'])

df.show()


+--------+----+----+
VendorID|col2|col3|
+--------+----+----+
 1|null|null|
 2| 2|null|
 3|null|null|
 5|null|null|
 4|four| IV|
+--------+----+----+

In [20]:
def getboth(df):
  return df.filter(' and '.join([''+x+' is not null' for x in df.columns])), df.exceptAll(df.filter(' and '.join([''+x+' is not null' for x in df.columns])))

NotNull, complete= getboth(df)
NotNull.show()
complete.show()

+----+----+----+
col1|col2|col3|
+----+----+----+
 4|four| IV|
+----+----+----+

+----+----+----+
col1|col2|col3|
+----+----+----+
null|null|null|
 3|null|null|
 1| one|null|
null| 2|null|
+----+----+----+

In [21]:
df.filter((' or '.join([''+x+' is not null' for x in df.columns if x !='VendorID'])))

+--------+----+----+
VendorID|col2|col3|
+--------+----+----+
 2| 2|null|
 4|four| IV|
+--------+----+----+

In [22]:
df.dropna(how='all', subset=[x for x in df.columns if x!='VendorID'])

+--------+----+----+
VendorID|col2|col3|
+--------+----+----+
 2| 2|null|
 4|four| IV|
+--------+----+----+

In [23]:
df.filter(' and '.join([''+x+' is not null' for x in df.columns])).show()

+----+----+----+
col1|col2|col3|
+----+----+----+
null|null|null|
+----+----+----+

In [24]:
" and ".join([""+x+" is not null" for x in df.columns])

Out[70]: 'col1 is not null and col2 is not null and col3 is not null'

In [25]:
' & '.join(['F.col('+x+').isNotNull()' for x in df.columns])

Out[48]: 'F.col(col1).isNotNull() & F.col(col2).isNotNull() & F.col(col3).isNotNull()'

In [26]:
df1=df.dropna(how="any")

df1.show()


+----+----+----+
col1|col2|col3|
+----+----+----+
 4|four| IV|
+----+----+----+

In [27]:
df.exceptAll(df1).show()

+----+----+----+
col1|col2|col3|
+----+----+----+
null|null|null|
 3|null|null|
 1| one|null|
null| 2|null|
+----+----+----+